<h1>Table of Contents<span class="tocSkip"></span></h1>
# <div class="toc"><ul class="toc-item"><li><span><a href="#一些套件" data-toc-modified-id="一些套件-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>一些套件</a></span></li><li><span><a href="#資料增強跟讀取" data-toc-modified-id="資料增強跟讀取-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>資料增強跟讀取</a></span></li><li><span><a href="#損失函數" data-toc-modified-id="損失函數-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>損失函數</a></span></li><li><span><a href="#模型" data-toc-modified-id="模型-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>模型</a></span></li></ul></div>

<input type="checkbox"> U-NET++  
<input type="checkbox"> TTA  
<input type="checkbox"> Pre-train model  

一張圖片切成 22 張 256 * 256 滑動 64，
每張一子圖做 8 次 TTA

其他
- 搞不好可以轉灰階

備註
- 訓練數據上 label 並沒有重疊
- 先把 mask 切好?

## 一些套件

In [2]:
!pip install tensorflow-gpu
!pip install keras
!pip install opencv-python
!pip install matplotlib
!pip install pandas
!pip install -U segmentation-models
# !pip install keras --upgrade

     |################################| 380.8MB 101kB/s  eta 0:00:01    |#########                       | 114.7MB 1.5MB/s eta 0:02:57     |#########################       | 309.2MB 28.2MB/s eta 0:00:03     |############################    | 339.2MB 24.8MB/s eta 0:00:02
     |################################| 51kB 22.3MB/s eta 0:00:01
     |################################| 61kB 36.9MB/s eta 0:00:01
     |################################| 51kB 33.5MB/s eta 0:00:01
     |################################| 2.3MB 43.6MB/s eta 0:00:01
     |################################| 1.3MB 46.0MB/s eta 0:00:01
     |################################| 450kB 36.6MB/s eta 0:00:01
     |################################| 71kB 37.4MB/s eta 0:00:01
     |################################| 3.8MB 35.9MB/s eta 0:00:01
     |################################| 112kB 61.4MB/s eta 0:00:01
     |################################| 2.9MB 38.1MB/s eta 0:00:01
     |################################| 327kB 50.2MB/s eta 0:00

In [1]:
import matplotlib.pyplot as plt
import random
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
# print(os.listdir('/home'))

In [2]:
# os.chdir('../input/severstal-steel-defect-detection')

train = pd.read_csv('train.csv')
train['ImageId_ClassId_1'] = train['ImageId_ClassId'].apply(lambda t :t[:-2])
train.set_index('ImageId_ClassId_1',inplace=True)

train.EncodedPixels[train.EncodedPixels.isnull()]=-1

In [3]:
# 查看異常比例
train[train.EncodedPixels!=-1].ImageId_ClassId.apply(lambda t: t[-1:]).value_counts()

3    5150
1     897
4     801
2     247
Name: ImageId_ClassId, dtype: int64

In [4]:
base_path='train_images'
batch_size = 32
df = train
img_list = os.listdir('train_images/')
img_list_random = np.random.permutation(img_list)
train_img, val_img = img_list_random[:10000], img_list_random[10000:]
n_classes = 5

In [5]:
batch_size = 32
df = train
width = 1600
height = 256
classes = 5
dim = 3

## 資料增強跟讀取

In [6]:
def random_argument(img,mask):
    # 隨機 256 x 256 的圖片
#     c = np.random.choice(1600-256)
    flip = np.random.choice([0,1])
#     rot = np.random.choice([0,1,2,3])
    rot = np.random.choice([0,2])
#     img = img[:,c:c+256]
#     mask = mask[:,c:c+256]
    img = np.rot90(img,rot)
    mask = np.rot90(mask,rot)
    if flip == 1:
        img = np.fliplr(img)
        mask = np.fliplr(mask)
    return img,mask

def rle2mask(rle, imgshape):
    
    if rle!=-1:
#         print('a')
        width = imgshape[0]
        height= imgshape[1]

        mask= np.zeros( width*height ).astype(np.uint8)

        array = np.asarray([int(x) for x in rle.split()])
        starts = array[0::2]
        lengths = array[1::2]

        for index, start in enumerate(starts):
            mask[int(start):int(start+lengths[index])] = 1

        return np.flipud( np.rot90( mask.reshape(height,width), k=1 ) )
    else:
        return np.zeros((256,1600))

def build_masks(rles, input_shape):
#     depth = len(rles)+1
    depth = len(rles)
    height, width = input_shape
    masks = np.zeros((height, width, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, (height, width))
#     # 加入正常鋼鐵類別
#     masks[:, :, depth-1] = -masks.sum(axis=2)+1
    
    return masks

def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0]
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i])
            for i in range(depth-1)] # 不計算第五類(正常鋼鐵) 
    return rles



## 損失函數

In [7]:
def Tversky_Loss(y_true, y_pred, smooth = 1, alpha = 0.3, beta = 0.7, flatten = False):
    
    if flatten:
        y_true = K.flatten(y_true)
        y_pred = K.flatten(y_pred)
    
    TP = K.sum(y_true * y_pred)
    FP = K.sum((1-y_true) * y_pred)
    FN = K.sum(y_true * (1-y_pred))
    
    tversky_coef = (TP + smooth) / (TP + alpha * FP + beta * FN + smooth)
    
    return 1 - tversky_coef

def Focal_Loss(y_true, y_pred, alpha = 0.8, gamma = 2.0, flatten = False):
    
    if flatten:
        y_true = K.flatten(y_true)
        y_pred = K.flatten(y_pred)    
    
    bce = keras.losses.binary_crossentropy(y_true, y_pred)
    bce_exp = K.exp(-bce)
    
    loss = K.mean(alpha * K.pow((1-bce_exp), gamma) * bce)
    return loss

def weighted_bce(weight = 0.6):
    
    def convert_2_logits(y_pred):
        y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1 - K.epsilon())
        return tf.log(y_pred / (1-y_pred))
    
    def weighted_binary_crossentropy(y_true, y_pred):
        y_pred = convert_2_logits(y_pred)
        loss = tf.nn.weighted_cross_entropy_with_logits(logits = y_pred, targets = y_true, pos_weight = weight)
        return loss
    
    return weighted_binary_crossentropy

def Combo_Loss(y_true, y_pred, a = 0.4, b = 0.2, c= 0.4):
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    return a*weighted_bce()(y_true, y_pred) + b*Focal_Loss(y_true_f, y_pred_f) + c*Tversky_Loss(y_true_f, y_pred_f)

def Dice_coef(y_true, y_pred, smooth = 1):

    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
 
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def Dice_loss(y_true, y_pred):   
    return  1.0 - Dice_coef(y_true, y_pred)

## 模型

In [8]:
import segmentation_models as sm
import keras.backend as K
from keras.callbacks import ModelCheckpoint,EarlyStopping

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [9]:
BACKBONE = 'efficientnetb5'
preprocess_input = sm.get_preprocessing(BACKBONE)
def build_masks(rles, input_shape):
    depth = len(rles)
    height, width = input_shape
    masks = np.zeros((height, width, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, (height, width))
    # 加入正常鋼鐵類別
#     masks[:, :, depth-1] = -masks.sum(axis=2)+1
    
    return masks

def data_generator(ID_list, width = 1600, height = 256, classes = 4,
                   dim = 3, batch_size=4):
    while True:

        choice_img = np.random.choice(ID_list,size=batch_size) # 因資料增強，不須抽出不放回
        df = train.loc[choice_img]
        X = np.empty((batch_size,height,1600,dim))
        Target = np.empty((batch_size,height,1600,classes))
        for i, img_jpg in enumerate(choice_img):
            rles = train.loc[img_jpg,'EncodedPixels']
            img = cv2.imread('train_images/'+img_jpg)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # 轉成 PGB
#             img = img.astype(np.float32) / 255. # 標準化
            img = preprocess_input(img)
            mask = build_masks(rles,(height,width))
            img,mask = random_argument(img,mask)
#             print(mask.shape,img.shape)
            X[i,] = img
            Target[i,] = mask

        yield X,Target

In [10]:

model = sm.FPN(BACKBONE, input_shape=(256, 1600, 3), 
               encoder_weights='imagenet',
               classes=4, activation='sigmoid')
model.compile(
    'Adam',
    loss=sm.losses.binary_focal_dice_loss,
    metrics=[Dice_coef,sm.metrics.IOUScore(),sm.metrics.f1_score],
)

In [14]:

train_generator = data_generator(train_img,batch_size=4)
val_generator = data_generator(val_img,batch_size=4)
z = next(train_generator)

In [15]:
z[1].shape

(4, 256, 1600, 4)

In [ ]:
checkpoint = ModelCheckpoint(
    'model_10241248.h5', 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)


history = model.fit_generator(
    train_generator, steps_per_epoch=100, epochs=2000000,
    validation_data=val_generator, validation_steps=50,
    callbacks=[checkpoint,EarlyStopping(patience=20)],
    use_multiprocessing=True,
    workers=-1
)

Epoch 1/2000000
100/100 [==============================] - 176s 2s/step - loss: 1.0415 - Dice_coef: 0.1879 - iou_score: 0.0444 - f1-score: 0.0706 - val_loss: 1.1011 - val_Dice_coef: 0.1981 - val_iou_score: 0.0411 - val_f1-score: 0.0657

Epoch 00001: val_loss improved from inf to 1.10113, saving model to model_10241248.h5
Epoch 2/2000000
100/100 [==============================] - 166s 2s/step - loss: 0.9595 - Dice_coef: 0.2341 - iou_score: 0.0641 - f1-score: 0.0993 - val_loss: 1.0926 - val_Dice_coef: 0.2052 - val_iou_score: 0.0618 - val_f1-score: 0.0969

Epoch 00002: val_loss improved from 1.10113 to 1.09259, saving model to model_10241248.h5
Epoch 3/2000000
100/100 [==============================] - 169s 2s/step - loss: 0.9621 - Dice_coef: 0.2501 - iou_score: 0.0697 - f1-score: 0.1063 - val_loss: 0.9373 - val_Dice_coef: 0.2466 - val_iou_score: 0.0571 - val_f1-score: 0.0897

Epoch 00003: val_loss improved from 1.09259 to 0.93730, saving model to model_10241248.h5
Epoch 4/2000000
100/100

In [12]:
history2 = model.fit_generator(
    train_generator, steps_per_epoch=1000, epochs=2000000,
    validation_data=val_generator, validation_steps=500,
    callbacks=[checkpoint,EarlyStopping(patience=20)],
    use_multiprocessing=True,
    workers=-1
)

Epoch 1/2000000
1000/1000 [==============================] - 548s 548ms/step - loss: 0.9143 - Dice_coef: 0.3627 - iou_score: 0.0975 - f1-score: 0.1355 - val_loss: 1.0692 - val_Dice_coef: 0.3609 - val_iou_score: 0.0874 - val_f1-score: 0.1197

Epoch 00001: val_loss did not improve from 0.82654
Epoch 2/2000000
1000/1000 [==============================] - 546s 546ms/step - loss: 0.9141 - Dice_coef: 0.3637 - iou_score: 0.0972 - f1-score: 0.1345 - val_loss: 1.0112 - val_Dice_coef: 0.3666 - val_iou_score: 0.0885 - val_f1-score: 0.1247

Epoch 00002: val_loss did not improve from 0.82654
Epoch 3/2000000
1000/1000 [==============================] - 547s 547ms/step - loss: 0.9104 - Dice_coef: 0.3771 - iou_score: 0.1000 - f1-score: 0.1378 - val_loss: 0.6990 - val_Dice_coef: 0.3661 - val_iou_score: 0.0997 - val_f1-score: 0.1375

Epoch 00003: val_loss improved from 0.82654 to 0.69899, saving model to model_10231756.h5
Epoch 4/2000000
1000/1000 [==============================] - 545s 545ms/step - los

In [11]:
history2 = model.fit_generator(
    train_generator, steps_per_epoch=100, epochs=2000000,
    validation_data=val_generator, validation_steps=50,
    callbacks=[checkpoint,EarlyStopping(patience=10)],
    use_multiprocessing=True,initial_epoch=26,
    workers=-1
)

Epoch 27/2000000
100/100 [==============================] - 284s 3s/step - loss: 0.7246 - Dice_coef: 0.9258 - iou_score: 0.2448 - val_loss: 0.7091 - val_Dice_coef: 0.9446 - val_iou_score: 0.2571

Epoch 00027: val_loss did not improve from 0.52859
Epoch 28/2000000
100/100 [==============================] - 286s 3s/step - loss: 0.7239 - Dice_coef: 0.9370 - iou_score: 0.2454 - val_loss: 0.7996 - val_Dice_coef: 0.9237 - val_iou_score: 0.2598

Epoch 00028: val_loss did not improve from 0.52859
Epoch 29/2000000
100/100 [==============================] - 284s 3s/step - loss: 0.7101 - Dice_coef: 0.9507 - iou_score: 0.2603 - val_loss: 0.6247 - val_Dice_coef: 0.9110 - val_iou_score: 0.2381

Epoch 00029: val_loss did not improve from 0.52859
Epoch 30/2000000
100/100 [==============================] - 285s 3s/step - loss: 0.7261 - Dice_coef: 0.9381 - iou_score: 0.2436 - val_loss: 0.8105 - val_Dice_coef: 0.9208 - val_iou_score: 0.2355

Epoch 00030: val_loss did not improve from 0.52859
Epoch 31/200

In [14]:
from keras.models import model_from_json

In [17]:
json_string = model.to_json()

In [19]:
import json

# Writing JSON data
with open('model_05.json', 'w') as f:
    json.dump(json_string, f)